In [1]:
from urllib.request import urlopen
import json
import time
from datetime import datetime, date, timedelta
import pandas as pd
from pandas import DataFrame, read_csv

In [2]:
def get_dates_and_times(mode=1, d1=None, d2=None, a_of_data=1000, a_of_date=None):
    '''Create date tuple between d1(beginning date) and d2(ending date).
    
    Mode 1: input d1 and the a_of_data to be downloaded
    Mode 2: input d1 and the a_of_date considered
    Mode 3: input d1 and d2
    Mode 4: input a_of_data, automatically start from the last date detected in the csv file
    '''
    
    data = pd.read_csv('weather_data_backup.csv')
    csv_date = data.date.tolist()
    csv_time = data.time.tolist()
    # date_today = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d')
    
    def mode_2_3(d1, delta):
        
        dates_and_times = ()
        
        for i in range(delta):
            time_tuple = (d1 + timedelta(days=i)).timetuple()
            for j in range(24):
                point_five_hour = (time_tuple[0], time_tuple[1], time_tuple[2], j, 30, 0, 0, 1)
                dates_and_times += int(time.mktime(point_five_hour.timetuple),)
                
        return dates_and_times
            
    def mode_1_4(d1, a_of_data):
        
        i = 1
        d_increment = 0
        h_increment = 0
        dates_and_times = ()
        
        while i <= a_of_data:
            
            time_tuple = (d1 + timedelta(days=d_increment)).timetuple()
            time_stamp = int(time.mktime(time_tuple))
            
            current_date = str(datetime.fromtimestamp(time_stamp).strftime('%Y-%m-%d'))
            current_time = '{}:30:00'.format(str(h_increment).zfill(2))

            checker = True
            for c_d, c_t in zip(csv_date, csv_time):

                if current_date == c_d and current_time == c_t: 
                    
                    checker = False
                    
            # if current_date == date_today: checker = False
            
            if checker:
                
                point_five_hour = (time_tuple[0], time_tuple[1], time_tuple[2], h_increment, 30, 0, 0, 0, 1)
                dates_and_times += (int(time.mktime(point_five_hour)),)

                i += 1
            
            h_increment += 1
            
            if h_increment == 24:
                h_increment = 0
                d_increment += 1
                
        return dates_and_times
    
    
    if mode == 1: 
        dates_and_times = mode_1_4(d1, a_of_data)
        
    if mode == 2: 
        dates_and_times = mode_2_3(d1, a_of_date)
    
    if mode == 3: 
        delta = (d2 - d1).days + 1
        dates_and_times = mode_2_3(d1, delta)
        
    if mode == 4: 
        temp = (data['date'][len(data)-1]).split('-')
        d1 = date(int(temp[0]), int(temp[1]), int(temp[2]))
        dates_and_times = mode_1_4(d1, a_of_data)
            
    print(str(len(dates_and_times)) + " times of download will be executed.")
    print('-' * 100)
        
    return dates_and_times

In [3]:
def check_duplication(date, time):
    '''Check if there is the same item in the csv file 'weather_data_backup.csv'.'''
    
    csv_file_name = 'weather_data_backup.csv'
    
    data = pd.read_csv(csv_file_name)
    csv_date = data.date.tolist()
    csv_time = data.time.tolist()
    
    # Empty csv file will trigger the importation immediately
    if not csv_date and not csv_time:
        return True
    
    checker = True
    
    # Loop through the csv file, one row with the same date and the same time will return False
    for c_d, c_t in zip(csv_date, csv_time):
        if date == c_d and time == c_t:
            checker = False
    
    if checker:
        return True
    else:
        return False


In [4]:
def save_to_csv(data_list):
    '''
    Add new data to the csv file 'weather_data_backup.csv' 
    and sorted data to 'weather_data.csv'.
    '''
    
    df = pd.DataFrame(data=data_list)
    
    with open('weather_data_backup.csv','a') as f:
        df.to_csv(f, header=False, index=False)
        
    # Sort data in the csv file
    data = pd.read_csv('weather_data_backup.csv')
    sorted_data = data.sort_values(['date', 'time'])
    df = pd.DataFrame(data=sorted_data)
    headers = df.head
    
    with open('weather_data.csv', 'w') as f:
        df.to_csv(f, header=headers, index=False)

In [5]:
def update_progress(desc, progress, speed):
    '''Progress indicator'''
    
    import time, sys
    
    length = 50
    block = int(round(length*progress))
    progress_str = '%0.1f'%(progress*100)
    
    progress_len = 5
    progress_str = ' ' * (progress_len - len(progress_str)) + progress_str
    
    if block <= 1:
        msg = "\r{0}: [{1}] {2}% | Spd: {3}s/date |".format(desc, ">" * block + "." * (length-block), 
                                                     progress_str, speed)
    if 1 < block < 50:
        msg = "\r{0}: [{1}] {2}% | Spd: {3}s/date |".format(desc, "=" * (block - 1) + '>' + "." * (length-block), 
                                                     progress_str, speed)
    
    
    if progress >= 1: msg = "\r{0}: [{1}] {2}% | Spd: {3}s/date | DONE\r\n".format(desc, "=" * (block) + "." * (length-block), 
                                                                              progress_str, speed)

    sys.stdout.write(msg)
    sys.stdout.flush()

In [6]:
def from_json_to_csv(converted_dates_and_times):
    '''Download the json file for a specific hour and store it in a local csv file.'''
    
    api_key = '3fa6b62f319752474af8fc227335a5cc'
    longitude_and_latitude = '48.864716,2.349014'
    raw_api = 'https://api.darksky.net/forecast/' + api_key + '/' + longitude_and_latitude + ',{}' + '?units=si'
    
    outer_time_start = time.time()
    
    date_today = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d')
    
    # Data headers wanted
    data_headers = ['temperature', 
                    'apparentTemperature', 
                    'precipIntensity', 
                    'precipProbability',
                    'dewPoint', 
                    'humidity', 
                    'pressure', 
                    'windSpeed', 
                    'cloudCover']
    
    previous_date = ''
    Y_m_d = ()
    H_M_S = ()
    error_messages = ()
    
    for i in converted_dates_and_times:
        Y_m_d += (str(datetime.fromtimestamp(i).strftime('%Y-%m-%d')),)
    
    # Main downloading and storing loop
    for time_stamp in converted_dates_and_times:
        
        inner_start_time = time.time()
        
        current_date = str(datetime.fromtimestamp(time_stamp).strftime('%Y-%m-%d'))
        current_time = str(datetime.fromtimestamp(time_stamp).strftime('%H:%M:%S'))
        current_hour = str(datetime.fromtimestamp(time_stamp).strftime('%H'))
        
        if current_date == date_today:
            print('Today is covered in the importation range, so the application is automatically stopped.')
            break
        
        if current_date != previous_date:
            iteration_count = 1
            n_iteration = Y_m_d.count(current_date)
            previous_date = current_date     
        else:
            iteration_count += 1

        if check_duplication(current_date, current_time):
            
            # Get json data from Darksky's API
            api = raw_api.format(str(time_stamp))
            # Handle the exception returned from the API
            try:
                json_data = json.loads(urlopen(api).read().decode())
            except:
                print('\n')
                print('Maximum calls per day is attained!')
                break
                
            filtered_data = []
        
            # Loading the whole forecast for a day at 0 o'clock of that day
            if current_hour == '00':
                
                for hourly_data in json_data['hourly']['data']:
                    temp = []
                    temp += ('forecast',)
                    temp += (datetime.fromtimestamp(hourly_data['time']).strftime('%Y-%m-%d'),)
                    temp += (datetime.fromtimestamp(hourly_data['time']).strftime('%H:%M:%S'),)
                    
                    for target in data_headers:
                        temp += (hourly_data.get(target),)
                    filtered_data.append(temp)
                    
            temp = []
            temp += ('now',)
            temp += (current_date,)
            temp += (current_time,)
            
            for target in data_headers:
                temp += ((json_data['currently']).get(target),)
            filtered_data.append(temp)
            save_to_csv(filtered_data)
            inner_elapsed_time = time.time() - inner_start_time
            
        else:
            error_messages += (current_date + ' ' + current_time + ' data already exists!',)
            inner_elapsed_time = time.time() - inner_start_time
            
        if iteration_count == 1:
            cumulated_time = inner_elapsed_time
            importation_speed = round(cumulated_time, 2)
        else:
            cumulated_time += inner_elapsed_time
            importation_speed = round(cumulated_time / iteration_count, 2)
            
        update_progress(current_date, iteration_count/n_iteration, '%0.2f'%(importation_speed))
    
    print('-' * 100)
    
    for i in error_messages:
        print(i)
        
    if error_messages == ():
        print('No error is recognized.')
        
    # Display summary of the importation
    print('-' * 100)
    print('Summary')
    
    outer_elapsed_time = time.time() - outer_time_start
    m, s = divmod(outer_elapsed_time, 60)
    
    print('Total time consumed: {} minutes {} seconds'.format(round(m), round(s)))
    data = pd.read_csv('weather_data.csv')
    print('Total amount of data in the csv file: {}'.format(len(data)))
    print('Current range of data: from {} to {}'.format(data['date'][0], data['date'][len(data) - 1]))

In [7]:
converted_dates_and_times = get_dates_and_times(mode=1, d1=date(2014,8, 8), a_of_data=1000)
from_json_to_csv(converted_dates_and_times)

1000 times of download will be executed.
----------------------------------------------------------------------------------------------------
2014-06-27: [==================================================] 100.0% | Spd: 1.49s/date | DONE
2014-06-28: [==================================================] 100.0% | Spd: 1.47s/date | DONE
2014-06-29: [==================================================] 100.0% | Spd: 1.83s/date | DONE
2014-06-30: [==================================================] 100.0% | Spd: 1.52s/date | DONE
2014-07-01: [==================================================] 100.0% | Spd: 1.50s/date | DONE
2014-07-02: [==================================================] 100.0% | Spd: 1.47s/date | DONE
2014-07-03: [==================================================] 100.0% | Spd: 1.51s/date | DONE
2014-07-04: [==================================================] 100.0% | Spd: 1.53s/date | DONE
2014-07-05: [==================================================] 100.0% | Spd: 1.6